# ADVANCE CNN PROJECT 

* Before moving forward to rnn let me introduce you all to advance cnn

* This is gonna be one good level code with explanation 

# About Project : -

1. deepfake detection model , it detect fake images by real images this is the important point  , WE GONNA USE VIT TRANSFORMERS

2. VIT with multi -scale Fusion ->  multi-scale attention with pretrained VIT  that focus oon artifacts of deepfake 

3. Multi - Scale Inference : 3 scales (224,256,192)  for robustness

4. Mixed Pecision + Gradient Accumulation : for 8gb vram optimized

5. Dropout + Adaptive LR : for overfitting and convergence 


# what is VIT ?

* Transformer - based image model , that uses patches and self-attention 


 * Example Output : - 


- 2025-08-12 19:20:00,123 - __main__ - INFO - Using device: cuda
- 2025-08-12 19:20:05,456 - __main__ - INFO - Initialized train dataset with 3500 images
...
- 2025-08-12 19:30:00,123 - __main__ - INFO - Epoch 25/25, Loss: 0.0756
- 2025-08-12 19:30:00,456 - __main__ - INFO - Validation Accuracy: 95.80%
- 2025-08-12 19:30:00,789 - __main__ - INFO - Test Accuracy: 95.60%
- 2025-08-12 19:30:01,012 - __main__ - INFO - Multi-Scale Prediction: 0
Final Prediction: Real



In [6]:
%pip install torch torchvision opencv-python-headless transformers

Using cached transformers-4.55.0-py3-none-any.whl (11.3 MB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.4 MB/s eta 0:00:02
   ------------ --------------------------- 0.8/2.5 MB 1.5 MB/s eta 0:00:02
   -------------------- ------------------- 1.3/2.5 MB 1.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.6/2.5 MB 1.5 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.5 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.5 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# EXPLAINING LIBRARIES: 

1. import torch.nn as nn -> neural network building blocks : layets(Linear , Conv3d , BatchNorm), activation , loss functions 

one use nn.Module to define cutom module architectures

2. import torch.optim as optim -> optimizers algorithm like Adam , SGD , AdamW to update model weights

Links model gradient from backward() to actual parameter updates 

3. from transformers import ViTModel , ViTConfig 

- ViTModel - > pretrained vision transformer backbone - handkes patch embedding , attention layers , position encoding 

- ViTConfig -> Defines architecture params (hidden size , attention heads , etc) if building from scratch or customization 

4. import torchvision.transformers as transformers -> Image data augmentation and normalization pipeline 

- Commonly chained in transforers.Composer() for resizing , cropping , normalization , tensor conversion

5. from torch.utils.data import Dataset , DataLoader -> 

- Dataset - Custom class interface for how your data is stored and retrieved 

- DataLoader - handles batching , shuffling and multi-threaded loading for efficient training 

6. import torch.cuda.amp import autocast , GradeScaler

- autocast - Automatically chooses FP16 OR FP32 operations to reduce memory and imporve training speed

- GradScaler - prevents underfloe when training in mixed precision by scalling loss before backward()

7. from torch.optim.lr_scheduler import OneCycleLR - adjning rate dynamically over training asteps according to the 1 cycle policy (warm-up , ramp-up , decay )

- helps models coverge faster an acoid local minima

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import ViTModel, ViTConfig
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import numpy as np
import logging
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.model_selection import train_test_split

# logging setup

logging.basicConfig(level = logging.INFO , format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                   
                   handlers = [logging.StreamHandler(), logging.FileHandler('deepfake_detection.log')]
                    )

logger = logging.getLogger(__name__)

class Config:
    def __init__(self):
        self.data_dir = "./deepfake_dataset"
        self.classes = ['real','deepfake']
        self.train_split = 0.7
        self.val_split = 0.2
        self.test_split = 0.1
        self.batch_size = 2
        self.patch_size = 16
        self.img_size = 224
        self.accumulation_steps = 2
        self.epochs = 25
        self.attention_heads = 16
        self.model_path ='deepfake_vit_model.pt'
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.lr= 0.0001
        self.max_lr = 0.0015
        self.weight_decay = 1e-4 
        self.dropout_rate = 0.2
        logger.info(f"Using device {self.device}")
        
        
        

## Vision Transformer Parameters Explained (Simplified)

- **Patch Size = 16**  
  The image is split into small squares of `16×16` pixels.  
  For a `224×224` image:  
  (224 / 16) × (224 / 16) = 14 × 14 = 196 patches

**196 patches** means the model sees the image as **196 small images**.

- **Gradient Accumulation Steps = 2**  
Instead of updating the model after every batch, we wait for **2 batches** before updating.  
This makes the *effective batch size* bigger without needing more GPU memory.

- **Attention Heads = 16**  
The transformer looks at the image patches from **16 different perspectives** at the same time.  
This helps it notice different types of patterns and relationships.

- **Model Path = 'deepfake_vit_model.pt'**  
Where the trained model is saved so we can use it later without retraining.

- **Weight Decay = 1e-4**  
Stops the model's weights from growing too big (reduces overfitting).  
Think of it like a small “shrink” effect on weights during training.
